In [1]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [2]:
%%capture
import datasets

import ipywidgets as widgets
from IPython.display import HTML, Markdown as md
import itertools

from lib.memory import DSDM
from lib.utils import cleanup, configs, inference, learning, utils 

import math
import numpy as np
import random

import pandas as pd
import pathlib
import pickle

import string


### Package options ###
pd.set_option('display.max_rows', 500)

In [3]:
# Set seed.
utils.fix_seed(41)

Using seed: 41

In [4]:
### Utils ###
def get_results(
    sentences: list[str],
    retrieve_mode: str,
    remove_stopwords_inference: bool = True
) -> None:
    
    # Retrieve content from memory.
    retrieved_contents = inference.infer(
        memory.address_size,
        cleanup,
        memory,
        sentences,
        retrieve_mode=retrieve_mode,
        remove_stopwords=remove_stopwords_inference,
        k=7, 
    )
    
    if retrieve_mode == "top_k":
        sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 
    
        for s, addresses in zip(sentences, retrieved_contents):
            display(md(f"<ins>**Sentence:**</ins> {s}"))
            out_tables = []
            for a in addresses:
                address_sims_df = inference.get_similarities_to_atomic_set(
                    a, cleanup, k=11)
                out = widgets.Output()
                with out:
                    display(address_sims_df)
                out_tables.append(out)
            display(widgets.HBox(out_tables))
        return
    elif retrieve_mode == "pooling":  
        sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 

        for s, c in zip(sentences, retrieved_contents):
            sentence_sims_df = inference.get_similarities_to_atomic_set(
                c, cleanup)
            sentence_sims_df['sentence'] = [s] * len(sentence_sims_df)
            sims_df = pd.concat([sims_df, sentence_sims_df])

        sims_df = sims_df.sort_values(['sentence', 'similarity'], ascending=False) \
                         .set_index(['sentence', 'token'])

        display(sims_df)
        return
    else:  # unrecognized
        display(md("Unrecognized retrieval mode."))
        return

In [5]:
in_sentences = [
    """Blaine was reared in a Prohibition home, and while still a young girl, she became a very active participant at temperance meetings, where she won great favor for her songs and recitations.""",
    """In 1910, she was elected to the position of organizer and lecturer of the National WCTU.""",
    """Another feature of her work was the organization of temperance mass-meetings of Sunday-school children, usually preceded by a formal parade.""",
    """With all other games played, a victory over Everton had put United top of the group on nine points.""",
    """The 2022 FA Women's League Cup Final was the 11th final of the FA Women's League Cup, England's secondary cup competition for women's football teams and its primary league cup tournament.""",
    """In 2020 Mico's single 'igare' awarded as the best song of the summer in Kiss Summer Awards.""",
    """She collected the speech and words of Dublin city and donated her collection to the Department of Irish Folklore at University College, Dublin.""",
    """Traditional palyanytsya was baked from yeast dough.""",
   """First, hops were boiled in a pot, which was then poured into a makitra, to which sifted wheat flour was added.""",
     """ Jonathan Holland of ScreenDaily deemed the film to be "superbly directed by Palomero, who seems to have a special gift for seeing the world through children's eyes." """   
]

In [6]:
out_sentences = [
    """As the population of all of the towns grew, the need for better transportation between them also grew.""",
    """The construction of the line was the subject of a legal challenge.""",
    """The extremes, which can be fatal to healthy people within six hours, could affect hundreds of millions of people unused to such conditions.""",
    """Dangerous heat stress events will spread rapidly across the world as global heating continues.""",
    """Whether or not history will determine that we are living in an ever more divided culture, it certainly feels that way."""
]

In [7]:
# Parametrized cell.
filename = "2023-09-10 13-01-52-622408.pkl"
experiment_no = 1

In [8]:
# Load memory and associated cleanup.
cleanup = pickle.load(open(f"cleanups/method2/cleanup_{filename}",'rb'))
memory = pickle.load(open(f"memories/method2/memory_{filename}",'rb'))

In [9]:
display(md(f"# Mining Transfomer attention - Experiment {experiment_no}"))

# Mining Transfomer attention - Experiment 1

## Memory visualiztion
### Statistics

In [10]:
display(md(f"Number of trained articles: {len(memory.wiki_articles)}"))
display(md(f"Number of memory updates: {memory.n_updates}"))
display(md(f"Number of memory expansions: {memory.n_expansions}"))
display(md(f"Updates percentage: {round(memory.n_updates / (memory.n_updates + memory.n_expansions), 3)}%"))
display(md(f"Number of existing memory addresses: {len(memory.addresses)}"))

Number of trained articles: 5010

Number of memory updates: 234448

Number of memory expansions: 136896

Updates percentage: 0.631%

Number of existing memory addresses: 136896

### Addresses

In [11]:
# addresses = np.random.randint(0, len(memory.addresses), size=30)

# for address in addresses:
#     display(md(f"### <ins>Address {address}</ins>"))
#     display(md(f"Address **chunk score:** {memory.scores[address][0]}, **bin score:** {memory.scores[address][1]}"))
#     address_sims_df = inference.get_similarities_to_atomic_set(
#             memory.addresses[address],
#             cleanup,
#     )
#     display(address_sims_df)

## In-sample sentences
### W/ stop words in inference sentence
#### Closest addresses

In [12]:
get_results(in_sentences[:3], "top_k", False)

<ins>**Sentence:**</ins> Blaine was reared in a Prohibition home, and while still a young girl, she became a very active participant at temperance meetings, where she won great favor for her songs and recitations.

<ins>**Sentence:**</ins> In 1910, she was elected to the position of organizer and lecturer of the National WCTU.

<ins>**Sentence:**</ins> Another feature of her work was the organization of temperance mass-meetings of Sunday-school children, usually preceded by a formal parade.

#### Pooled address space

In [13]:
get_results(in_sentences[:3], "pooling", False)

similarity
sentence                                           token                
In 1910, she was elected to the position of org... the              0.60
                                                   of               0.51
                                                   to               0.26
                                                   wctu             0.26
                                                   in               0.24
                                                   and              0.20
                                                   was              0.19
                                                   a                0.16
                                                   claiming         0.12
                                                   diagnosed        0.12
Blaine was reared in a Prohibition home, and wh... the              0.44
                                                   a                0.38
                                                   of               0.38
                                                   in               0.31
                                                   and              0.28
                                                   was              0.21
                                                   to               0.19
                                                   wctu             0.19
                                                   an               0.14
                                                   claiming         0.13
Another feature of her work was the organizatio... of               0.64
                                                   the              0.48
                                                   a                0.26
                                                   in               0.21
                                                   was              0.19
                                                   and              0.18
                                                   to               0.15
                                                   wctu             0.15
                                                   claiming         0.13
                                                   by               0.13

### W/o stop words in inference sentence

In [14]:
get_results(in_sentences[:3], "top_k", True)

<ins>**Sentence:**</ins> Blaine was reared in a Prohibition home, and while still a young girl, she became a very active participant at temperance meetings, where she won great favor for her songs and recitations.

<ins>**Sentence:**</ins> In 1910, she was elected to the position of organizer and lecturer of the National WCTU.

<ins>**Sentence:**</ins> Another feature of her work was the organization of temperance mass-meetings of Sunday-school children, usually preceded by a formal parade.

In [15]:
get_results(in_sentences[:3], "pooling", True)

similarity
sentence                                           token            
In 1910, she was elected to the position of org... the          0.47
                                                   of           0.40
                                                   to           0.39
                                                   wctu         0.39
                                                   in           0.24
                                                   a            0.22
                                                   and          0.20
                                                   was          0.18
                                                   is           0.15
                                                   on           0.14
Blaine was reared in a Prohibition home, and wh... the          0.52
                                                   of           0.37
                                                   in           0.27
                                                   wctu         0.23
                                                   to           0.23
                                                   a            0.23
                                                   and          0.22
                                                   was          0.19
                                                   is           0.17
                                                   an           0.15
Another feature of her work was the organizatio... the          0.48
                                                   of           0.41
                                                   in           0.27
                                                   a            0.24
                                                   and          0.23
                                                   wctu         0.22
                                                   to           0.22
                                                   was          0.19
                                                   an           0.16
                                                   is           0.16

## Out-of-sample sentences
### W/ stop words in inference sentence
#### Closest addresses

In [16]:
get_results(out_sentences[:3], "top_k", False)

<ins>**Sentence:**</ins> As the population of all of the towns grew, the need for better transportation between them also grew.

<ins>**Sentence:**</ins> The construction of the line was the subject of a legal challenge.

<ins>**Sentence:**</ins> The extremes, which can be fatal to healthy people within six hours, could affect hundreds of millions of people unused to such conditions.

#### Pooled address space

In [17]:
get_results(out_sentences[:3], "pooling", False)

similarity
sentence                                           token                
The extremes, which can be fatal to healthy peo... of               0.53
                                                   the              0.50
                                                   to               0.29
                                                   wctu             0.29
                                                   a                0.27
                                                   extremes         0.27
                                                   in               0.21
                                                   and              0.18
                                                   was              0.17
                                                   an               0.14
The construction of the line was the subject of... the              0.72
                                                   of               0.50
                                                   a                0.19
                                                   extremes         0.19
                                                   in               0.17
                                                   was              0.16
                                                   and              0.13
                                                   bear             0.12
                                                   claiming         0.12
                                                   diagnosed        0.12
As the population of all of the towns grew, the... the              0.72
                                                   of               0.48
                                                   in               0.19
                                                   extremes         0.15
                                                   a                0.15
                                                   and              0.15
                                                   wctu             0.13
                                                   to               0.13
                                                   was              0.12
                                                   hamlet           0.12

### W/o stop words in inference sentence
#### Closest addresses

In [18]:
get_results(out_sentences[:3], "top_k", True)

<ins>**Sentence:**</ins> As the population of all of the towns grew, the need for better transportation between them also grew.

<ins>**Sentence:**</ins> The construction of the line was the subject of a legal challenge.

<ins>**Sentence:**</ins> The extremes, which can be fatal to healthy people within six hours, could affect hundreds of millions of people unused to such conditions.

#### Pooled address space

In [19]:
get_results(out_sentences[:3], "pooling", True)

similarity
sentence                                           token               
The extremes, which can be fatal to healthy peo... the             0.47
                                                   of              0.42
                                                   a               0.33
                                                   extremes        0.33
                                                   in              0.26
                                                   to              0.21
                                                   wctu            0.21
                                                   and             0.20
                                                   was             0.20
                                                   an              0.16
The construction of the line was the subject of... the             0.49
                                                   of              0.42
                                                   in              0.27
                                                   wctu            0.24
                                                   to              0.24
                                                   extremes        0.22
                                                   a               0.22
                                                   and             0.20
                                                   was             0.20
                                                   is              0.17
As the population of all of the towns grew, the... the             0.52
                                                   of              0.40
                                                   in              0.28
                                                   wctu            0.25
                                                   to              0.25
                                                   extremes        0.22
                                                   a               0.22
                                                   and             0.21
                                                   was             0.18
                                                   is              0.15

In [20]:
# # Set retrieve mode.
# retrieve_mode = "top_k"

# # Get table with token similarities for each "out-of-train" sentence.
# retrieved_contents = inference.infer(
#     memory.address_size,
#     cleanup,
#     memory,
#     sentences,
#     retrieve_mode=retrieve_mode,
#     remove_stopwords=True,
#     k=7, 
# )


# sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 

# for s, addresses in zip(sentences, retrieved_contents):
#     display(s)
#     out_tables = []
#     for a in addresses:
#         address_sims_df = inference.get_similarities_to_atomic_set(
#             a, cleanup, k=11)
#         out = widgets.Output()
#         with out:
#             display(address_sims_df)
#         out_tables.append(out)
#     display(widgets.HBox(out_tables))

In [21]:
# retrieve_mode = "pooling"

# # Get table with token similarities for each "out-of-train" sentence.
# retrieved_contents = inference.infer(
#     memory.address_size,
#     cleanup,
#     memory,
#     sentences,
#     retrieve_mode=retrieve_mode,
#     remove_stopwords=True,
#     k=7, 
# )

# sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 
      
# for s, c in zip(sentences, retrieved_contents):
#     sentence_sims_df = inference.get_similarities_to_atomic_set(
#         c, cleanup)
#     sentence_sims_df['sentence'] = [s] * len(sentence_sims_df)
#     sims_df = pd.concat([sims_df, sentence_sims_df])

# sims_df = sims_df.sort_values(['sentence', 'similarity'], ascending=False) \
#                  .set_index(['sentence', 'token'])

# display(sims_df)